In [14]:
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns

In [29]:
dataset=r"C:\Users\ananya\Desktop\SIH\student_dropout_1000_students.csv"

In [30]:
df = pd.read_csv(dataset)
df.head()

,student_id,sem_no,attendance_pct,new_agg_marks,dropout,pct_change,slope,momentum,fee_days
0,1,7,82.55,74.39,0,-1.22,0.1674,0.1654,17
1,2,2,86.05,70.14,0,-14.98,1.4844,1.2621,10
2,3,3,76.12,68.38,0,14.21,-0.0345,-0.0394,24
3,4,4,82.53,66.52,0,3.23,-0.1590,-0.1641,16
4,5,4,62.46,43.72,1,-0.14,-0.4312,-0.4306,18


In [31]:
#adding noise
import pandas as pd
import numpy as np

# 1. Load your original data
file_path = r"C:\Users\ananya\Desktop\SIH\student_dropout_1000_students.csv"
df = pd.read_csv(file_path)

# ---------------------------------------------------------
# CONFIGURATION
# ---------------------------------------------------------
NOISE_LEVEL = 0.05   # 5% variation (Adjust if needed)
TARGET_COLUMN = 'dropout' 
ID_COLUMN = 'student_id'
SKIP_COLS = ['sem_no'] # Columns we shouldn't add noise to
# ---------------------------------------------------------

# Create a copy so we don't mess up the original 'df' variable
df_noisy = df.copy()

# Select numerical columns
numeric_cols = df_noisy.select_dtypes(include=[np.number]).columns

print("Adding noise...")

for col in numeric_cols:
    # SKIP the ID, Target, and Semester columns
    if col in [TARGET_COLUMN, ID_COLUMN] + SKIP_COLS:
        continue
    
    # Calculate noise based on standard deviation
    std_dev = df_noisy[col].std()
    noise = np.random.normal(0, std_dev * NOISE_LEVEL, size=len(df_noisy))
    
    # Add noise to the column
    df_noisy[col] = df_noisy[col] + noise

# 2. SAVE to a new file
output_path = r"C:\Users\ananya\Desktop\SIH\student_dropout_1000_students_NOISY.csv"
df_noisy.to_csv(output_path, index=False)

print(f"File saved successfully to: {output_path}")

# 3. VERIFY: Show the first few rows of the NEW dataframe
print("\nFirst 5 rows of the new noisy dataframe (df_noisy):")
print(df_noisy.head())

Adding noise...
File saved successfully to: C:\Users\ananya\Desktop\SIH\student_dropout_1000_students_NOISY.csv

First 5 rows of the new noisy dataframe (df_noisy):
   student_id  sem_no  attendance_pct  new_agg_marks  dropout  pct_change  \
0           1       7       82.903806      73.822634        0   -1.540670   
1           2       2       86.128802      69.649848        0  -14.732039   
2           3       3       75.596840      68.120122        0   14.364746   
3           4       4       81.870619      66.618040        0    3.276867   
4           5       4       63.418263      42.977768        1   -0.402666   

      slope  momentum   fee_days  
0  0.234694  0.147869  16.503665  
1  1.435963  1.228247   9.865535  
2  0.030615  0.054164  24.246138  
3 -0.149150 -0.162519  15.932230  
4 -0.364225 -0.553659  17.300951  


In [32]:
X = df_noisy.drop(['student_id', 'dropout'], axis=1)
y = df_noisy['dropout']

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
model = XGBClassifier(
    n_estimators=100,       # Number of decision trees
    learning_rate=0.1,      # How fast the model learns
    max_depth=5,            # How complex the trees can get
    use_label_encoder=False,
    eval_metric='logloss'
)

model.fit(X_train, y_train)

C:\Users\ananya\Desktop\SIH\venv\Lib\site-packages\xgboost\training.py:199: UserWarning: [15:00:59] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


,"objective objective: typing.Union[str, xgboost.sklearn._SklObjWProto, typing.Callable[[typing.Any, typing.Any], typing.Tuple[numpy.ndarray, numpy.ndarray]], NoneType]Specify the learning task and the corresponding learning objective or a customobjective function to be used.For custom objective, see :doc:`/tutorials/custom_metric_obj` and:ref:`custom-obj-metric` for more information, along with the end note forfunction signatures.",'binary:logistic'
,"base_score base_score: typing.Union[float, typing.List[float], NoneType]The initial prediction score of all instances, global bias.",None
,booster,None
,"callbacks callbacks: typing.Optional[typing.List[xgboost.callback.TrainingCallback]]List of callback functions that are applied at end of each iteration.It is possible to use predefined callbacks by using:ref:`Callback API `... note:: States in callback are not preserved during training, which means callback objects can not be reused for multiple training sessions without reinitialization or deepcopy... code-block:: python for params in parameters_grid: # be sure to (re)initialize the callbacks before each run callbacks = [xgb.callback.LearningRateScheduler(custom_rates)] reg = xgboost.XGBRegressor(**params, callbacks=callbacks) reg.fit(X, y)",None
,colsample_bylevel colsample_bylevel: typing.Optional[float]Subsample ratio of columns for each level.,None
,colsample_bynode colsample_bynode: typing.Optional[float]Subsample ratio of columns for each split.,None
,colsample_bytree colsample_bytree: typing.Optional[float]Subsample ratio of columns when constructing each tree.,None
,"device device: typing.Optional[str].. versionadded:: 2.0.0Device ordinal, available options are `cpu`, `cuda`, and `gpu`.",None
,"early_stopping_rounds early_stopping_rounds: typing.Optional[int].. versionadded:: 1.6.0- Activates early stopping. Validation metric needs to improve at least once in every **early_stopping_rounds** round(s) to continue training. Requires at least one item in **eval_set** in :py:meth:`fit`.- If early stopping occurs, the model will have two additional attributes: :py:attr:`best_score` and :py:attr:`best_iteration`. These are used by the :py:meth:`predict` and :py:meth:`apply` methods to determine the optimal number of trees during inference. If users want to access the full model (including trees built after early stopping), they can specify the `iteration_range` in these inference methods. In addition, other utilities like model plotting can also use the entire model.- If you prefer to discard the trees after `best_iteration`, consider using the callback function :py:class:`xgboost.callback.EarlyStopping`.- If there's more than one item in **eval_set**, the last entry will be used for early stopping. If there's more than one metric in **eval_metric**, the last metric will be used for early stopping.",None
,enable_categorical enable_categorical: boolSee the same parameter of :py:class:`DMatrix` for details.,False
,"eval_metric eval_metric: typing.Union[str, typing.List[typing.Union[str, typing.Callable]], typing.Callable, NoneType].. versionadded:: 1.6.0Metric used for monitoring the training result and early stopping. It can be astring or list of strings as names of predefined metric in XGBoost (See:doc:`/parameter`), one of the metrics in :py:mod:`sklearn.metrics`, or anyother user defined metric that looks like `sklearn.metrics`.If custom objective is also provided, then custom metric should implement thecorresponding reverse link function.Unlike the `scoring` parameter commonly used in scikit-learn, when a callableobject is provided, it's assumed to be a cost function and by default XGBoostwill minimize the result during early stopping.For advanced usage on Early stopping like directly choosing to maximize insteadof minimize, see :py:obj:`xgboost.callback.EarlyStopping`.See :doc:`/tutorials/custom_metric_obj` and :ref:`custom-obj-metric` for moreinformation... code-block:: python from sklearn.datasets import load_diabetes 

In [35]:
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]
# Print Stats
accuracy = accuracy_score(y_test, y_pred)
print(f"\n📊 Model Accuracy: {accuracy:.2%}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


📊 Model Accuracy: 99.50%

Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       142
           1       1.00      0.98      0.99        58

    accuracy                           0.99       200
   macro avg       1.00      0.99      0.99       200
weighted avg       1.00      0.99      0.99       200

